In [1]:
import pandas as pd
import numpy as np
from numpy import pi
from scipy import signal
import os
import plotly.express as px
import plotly.io as pio

from plotly.subplots import make_subplots
pio.renderers.default = "plotly_mimetype+notebook"

In [2]:
def getBodypartDistance(dataframe, partname1, partname2):
    # retrieves the pixel distance between two bodyparts

    bpt1 = dataframe.xs(partname1, level='bodyparts', axis=1).to_numpy()
    bpt2 = dataframe.xs(partname2, level='bodyparts', axis=1).to_numpy()

    bptDistance = np.sqrt(np.sum(np.square(bpt1[:, [0, 1]] - bpt2[:, [0, 1]]), axis=1))
    return bptDistance


def getAnimalCentroid(dataframe):
    scorer = dataframe.columns.get_level_values(0).unique()[0]
    bptNames = dataframe.columns.get_level_values(1).unique()
    aniXpos, aniYpos = np.zeros((nFrames, len(bptNames[0:8]))), np.zeros((nFrames, len(bptNames[0:8])))

    for i, nm in enumerate(bptNames[0:8]):
        aniXpos[:, i] = dataframe[scorer][nm]['x']
        aniYpos[:, i] = dataframe[scorer][nm]['y']

    xCenter, yCenter = np.nanmean(aniXpos, axis=1), np.nanmean(aniYpos, axis=1)
    centroid = np.column_stack((xCenter, yCenter))

    return centroid


def getBodypartAngle(dataframe, partName1, partName2):
    # retrieves the angle between two bodyparts

    bpt1 = dataframe.xs(partName1, level='bodyparts', axis=1).to_numpy()
    bpt2 = dataframe.xs(partName2, level='bodyparts', axis=1).to_numpy()

    angle = np.arctan2(bpt2[:, 1] - bpt1[:, 1], bpt2[:, 0] - bpt1[:, 0])
    return angle


def circularDistance(ang1, ang2):
    # efficiently computes the circular distance between two angles

    circdist = np.angle(np.exp(1j * ang1) / np.exp(1j * ang2))

    return circdist


def wrapTo2pi(ang):
    # wraps a set of values to fit between zero and 2Pi

    positiveValues = (ang > 0)
    wrappedAng = ang % (2 * pi)
    wrappedAng[ang == 0 & positiveValues] = 2 * pi

    return wrappedAng


def getRelativeOrientations(ani1, ani2):

    normPos = ani2[['XCenter', 'YCenter']] - ani1[['XCenter', 'YCenter']]
    absoluteAngle = np.arctan2(normPos['YCenter'], normPos['XCenter'])
    fA = circularDistance(absoluteAngle, ani2['heading'])
    aBetween = circularDistance(ani1['heading'],ani2['heading'])

    return fA, aBetween


def removeJumps(dataframe, maxJumpLength):
    # removes large jumps in the x/y position of bodyparts, usually resulting from swaps between animals

    # get all column names
    scorer = dataframe.columns.get_level_values(0)[0]
    bps = list(dataframe.columns.levels[1])
    params = list(dataframe.columns.levels[2])
    dataframeMod = dataframe.copy()

    for i, partName in enumerate(bps):

        xDiff = pd.Series(np.diff(dataframe[scorer][partName]['x']))
        yDiff = pd.Series(np.diff(dataframe[scorer][partName]['y']))

        xJumpsPositive = signal.find_peaks(xDiff.interpolate(), threshold=200)
        xJumpsNegative = signal.find_peaks(xDiff.interpolate() * -1, threshold=200)
        yJumpsPositive = signal.find_peaks(yDiff.interpolate(), threshold=200)
        yJumpsNegative = signal.find_peaks(yDiff.interpolate() * -1, threshold=200)

        toKill = np.zeros((len(yDiff),), dtype=bool)

        for j in range(len(xJumpsPositive[0])):
            if np.any((xJumpsNegative[0] > xJumpsPositive[0][j]) & (
                    xJumpsNegative[0] < xJumpsPositive[0][j] + maxJumpLength)):
                endIdx = np.where((xJumpsNegative[0] > xJumpsPositive[0][j]) & (
                        xJumpsNegative[0] < xJumpsPositive[0][j] + maxJumpLength))
                toKill[xJumpsPositive[0][j]:xJumpsNegative[0][endIdx[0][0]]] = True
            else:
                toKill[xJumpsPositive[0][j]] = True

        for j in range(len(xJumpsNegative[0])):

            if np.any((xJumpsPositive[0] > xJumpsNegative[0][j]) & (
                    xJumpsPositive[0] < xJumpsNegative[0][j] + maxJumpLength)):
                endIdx = np.where((xJumpsPositive[0] > xJumpsNegative[0][j]) & (
                        xJumpsPositive[0] < xJumpsNegative[0][j] + maxJumpLength))
                toKill[xJumpsNegative[0][j]:xJumpsPositive[0][endIdx[0][0]]] = True
            else:
                toKill[xJumpsNegative[0][j]] = True

        for j in range(len(yJumpsPositive[0])):
            if np.any((yJumpsNegative[0] > yJumpsPositive[0][j]) & (
                    yJumpsNegative[0] < yJumpsPositive[0][j] + maxJumpLength)):
                endIdx = np.where((yJumpsNegative[0] > yJumpsPositive[0][j]) & (
                        yJumpsNegative[0] < yJumpsPositive[0][j] + maxJumpLength))
                toKill[yJumpsPositive[0][j]:yJumpsNegative[0][endIdx[0][0]]] = True
            else:
                toKill[yJumpsPositive[0][j]] = True

        for j in range(len(yJumpsNegative[0])):
            if np.any((yJumpsPositive[0] > yJumpsNegative[0][j]) & (
                    yJumpsPositive[0] < yJumpsNegative[0][j] + maxJumpLength)):
                endIdx = np.where((yJumpsPositive[0] > yJumpsNegative[0][j]) & (
                        yJumpsPositive[0] < yJumpsNegative[0][j] + maxJumpLength))
                toKill[yJumpsNegative[0][j]:yJumpsPositive[0][endIdx[0][0]]] = True
            else:
                toKill[yJumpsNegative[0][j]] = True

        toKill = np.insert(toKill, 1, False)

        dataframeMod.loc[toKill, (scorer, partName, params)] = np.nan

    return dataframeMod


def butterLowpassFilter(data, cutoff, fs, order):
    # implements a butterworth lowpass filter to a signal (data) sampled at fs, with a set cutoff.

    nyq = fs / 2
    adjustedCutoff = cutoff / nyq
    b, a = signal.butter(order, adjustedCutoff, btype='low', analog=False)
    filteredSignal = signal.filtfilt(b, a, data)

    return filteredSignal


def getContinousNumbers(nums):
    # returns a list of continous numbers found in an array

    nums = sorted(set(nums))
    gaps = [[s, e] for s, e in zip(nums, nums[1:]) if s + 1 < e]
    edges = iter(nums[:1] + sum(gaps, []) + nums[-1:])
    return list(zip(edges, edges))

In [3]:
folder_pathname = 'C:/Users/rutalab/Documents/Rufei_tmp/JAABAprep/MF'
os.chdir(folder_pathname)
folders = os.listdir(folder_pathname)
for folder in folders:
    if folder.startswith('.'):
        continue

    pathname = os.path.join(folder_pathname, folder)
    os.chdir(pathname)
    files = os.listdir(pathname)

    for file in files:
        if file.endswith('.h5'):
            filename = file

            #pathname = '/Users/rufeili/Documents/RutaLab/experimental data/temp/JAABA_prep/MF/DLC/030122_1'
            #filename = '030122_Canton-S_age5_m_sh_f_gh_1DLC_dlcrnetms5_MMFFeb15shuffle1_200000_el_IDcorrected.h5'

            tracks = pd.read_hdf(os.path.join(pathname, filename))
            scorer = tracks.columns.get_level_values(0)[0]
            sampleRate = 60  # Hz
            maxJump = 6
            tstamp = np.linspace(0, len(tracks) * 1 / sampleRate, len(tracks))
            nFrames = len(tracks)

            maleName = 'ind1' # double check in the plots for abdomen lengths
            femaleName = 'ind2'

            femalePositions = tracks.xs(femaleName, level='individuals', axis=1)
            female_abdomenlength = getBodypartDistance(femalePositions, 'abdomenTop', 'genitalia')
            malePositions = tracks.xs(maleName, level='individuals', axis=1)
            male_abdomenlength = getBodypartDistance(malePositions, 'abdomenTop', 'genitalia')


            if np.nanmean(female_abdomenlength) < np.nanmean(male_abdomenlength):
                maleName = 'ind2'
                femaleName = 'ind1'

            # get female positions
            femalePositions = tracks.xs(femaleName, level='individuals', axis=1)
            femalePositions = removeJumps(femalePositions, maxJump)
            femaleCenter = getAnimalCentroid(femalePositions)

            # get male positions
            malePositions = tracks.xs(maleName, level='individuals', axis=1)
            malePositions = removeJumps(malePositions, maxJump)
            maleCenter = getAnimalCentroid(malePositions)

            # detect copulation: cut dataframes
            copFrame = len(tracks)

            # get some more female parameters
            femaleParams = pd.DataFrame(getBodypartAngle(femalePositions, 'abdomenTop', 'thoraxCenter'),
                                        columns=['heading'])
            femaleParams['XCenter'], femaleParams['YCenter'] = femaleCenter[:copFrame, 0], femaleCenter[:copFrame, 1]
            femaleParams['linSpeed'] = np.concatenate(
                (np.zeros(1), np.sqrt(np.sum(np.square(np.diff(femaleCenter[:copFrame, ], axis=0)), axis=1))))
            leftW = getBodypartAngle(femalePositions, 'thoraxCenter', 'wingLeft')
            rightW = getBodypartAngle(femalePositions, 'thoraxCenter', 'wingRight')
            femaleParams['leftWingAngle'] = wrapTo2pi(circularDistance(femaleParams['heading'].interpolate(), leftW)) - np.pi
            femaleParams['rightWingAngle'] = wrapTo2pi(circularDistance(femaleParams['heading'].interpolate(), rightW)) - np.pi
            femaleParams['interWingDistance'] = getBodypartDistance(femalePositions, 'wingRight', 'wingLeft')
            femaleParams['abdomenLength'] = getBodypartDistance(femalePositions, 'abdomenTop', 'genitalia')
            femaleParams['abdomenWidth'] = getBodypartDistance(femalePositions, 'abdomenLeft', 'abdomenRight')
            femaleParams['abdomentipdirection'] = getBodypartAngle(femalePositions, 'genitalia', 'abdomenTop')
            femaleParams['abdomentipOffset'] = circularDistance(femaleParams['abdomentipdirection'],femaleParams['heading'])
            abdomen_lower_width = getBodypartDistance(femalePositions, 'abdomenLowerLeft', 'abdomenLowerRight')
            femaleParams['abdomenLowerwidth'] = abdomen_lower_width
            femaleParams['abdomenWidthRatio'] = abdomen_lower_width / femaleParams['abdomenWidth']
            femaleParams['abdomentipdirection'] = getBodypartAngle(femalePositions, 'genitalia', 'abdomenTop')
            femaleParams['abdomentipOffset'] = circularDistance(femaleParams['abdomentipdirection'],femaleParams['heading'])
            ableft = getBodypartAngle(femalePositions, 'genitalia','abdomenLowerLeft')
            abright = getBodypartAngle(femalePositions, 'genitalia','abdomenLowerRight')
            abmidline = getBodypartAngle(femalePositions, 'genitalia', 'abdomenCenter')
            abtip_angle = wrapTo2pi(circularDistance(abright,ableft))
            femaleParams['abdomen_tip_angle'] = abtip_angle
            abtip_left = wrapTo2pi(circularDistance(abmidline,ableft))
            abtip_right = wrapTo2pi(circularDistance(abright,abmidline))
            abtip_offset = abs(abtip_left - abtip_right)
            femaleParams['abdomentipOffset_m2'] = abtip_offset

            midleg_left_1 = getBodypartAngle(femalePositions, 'midlegLeftJoint1', 'thoraxCenter')
            midleg_right_1 = getBodypartAngle(femalePositions, 'midlegRightJoint1', 'thoraxCenter')
            midleg_left_2 = getBodypartAngle(femalePositions, 'midlegLeftJoint1', 'midlegLeftJoint2')
            midleg_right_2 = getBodypartAngle(femalePositions, 'midlegRightJoint1', 'midlegRightJoint2')

            midleg_left_Joint1_Ang = wrapTo2pi(circularDistance(midleg_left_1,midleg_left_2))
            midleg_right_Joint1_Ang = wrapTo2pi(circularDistance(midleg_right_1,midleg_right_2))
            femaleParams['midleg_left_Joint1_Ang'] = midleg_left_Joint1_Ang
            femaleParams['midleg_right_Joint1_Ang'] = midleg_right_Joint1_Ang

            midleg_left_2p = getBodypartAngle(femalePositions, 'midlegLeftJoint2', 'midlegLeftJoint1')
            midleg_right_2p = getBodypartAngle(femalePositions, 'midlegRightJoint2', 'midlegRightJoint1')
            midleg_left_3 = getBodypartAngle(femalePositions, 'midlegLeftJoint2', 'midlegLeft')
            midleg_right_3 = getBodypartAngle(femalePositions, 'midlegRightJoint2', 'midlegRight')

            midleg_left_Joint2_Ang = wrapTo2pi(circularDistance(midleg_left_2p,midleg_left_3))
            midleg_right_Joint2_Ang = wrapTo2pi(circularDistance(midleg_right_2p,midleg_right_3))
            femaleParams['midleg_left_Joint2_Ang'] = midleg_left_Joint2_Ang
            femaleParams['midleg_right_Joint2_Ang'] = midleg_right_Joint2_Ang

            hindleg_left_1 = getBodypartAngle(femalePositions, 'hindlegLeftJoint1', 'thoraxCenter')
            hindleg_right_1 = getBodypartAngle(femalePositions, 'hindlegRightJoint1', 'thoraxCenter')
            hindleg_left_2 = getBodypartAngle(femalePositions, 'hindlegLeftJoint1', 'hindlegLeftJoint2')
            hindleg_right_2 = getBodypartAngle(femalePositions, 'hindlegRightJoint1', 'hindlegRightJoint2')

            hindleg_left_Joint1_Ang = wrapTo2pi(circularDistance(hindleg_left_1,hindleg_left_2))
            hindleg_right_Joint1_Ang = wrapTo2pi(circularDistance(hindleg_right_1,hindleg_right_2))
            femaleParams['hindleg_left_Joint1_Ang'] = hindleg_left_Joint1_Ang
            femaleParams['hindleg_right_Joint1_Ang'] = hindleg_right_Joint1_Ang

            hindleg_left_2p = getBodypartAngle(femalePositions, 'hindlegLeftJoint2', 'hindlegLeftJoint1')
            hindleg_right_2p = getBodypartAngle(femalePositions, 'hindlegRightJoint2', 'hindlegRightJoint1')
            hindleg_left_3 = getBodypartAngle(femalePositions, 'hindlegLeftJoint2', 'hindlegLeft')
            hindleg_right_3 = getBodypartAngle(femalePositions, 'hindlegRightJoint2', 'hindlegRight')

            hindleg_left_Joint2_Ang = wrapTo2pi(circularDistance(hindleg_left_2p,hindleg_left_3))
            hindleg_right_Joint2_Ang = wrapTo2pi(circularDistance(hindleg_right_2p,hindleg_right_3))
            femaleParams['hindleg_left_Joint2_Ang'] = hindleg_left_Joint2_Ang
            femaleParams['hindleg_right_Joint2_Ang'] = hindleg_right_Joint2_Ang


            # get some more male parameters
            maleParams = pd.DataFrame(getBodypartAngle(malePositions, 'abdomenTop', 'thoraxCenter'),
                                      columns=['heading'])
            maleParams['XCenter'], maleParams['YCenter'] = maleCenter[:copFrame, 0], maleCenter[:copFrame, 1]
            maleParams['linSpeed'] = np.concatenate(
                (np.zeros(1), np.sqrt(np.sum(np.square(np.diff(maleCenter[:copFrame, ], axis=0)), axis=1))))
            leftW = getBodypartAngle(malePositions, 'thoraxCenter', 'wingLeft')
            rightW = getBodypartAngle(malePositions, 'thoraxCenter', 'wingRight')
            maleParams['leftWingAngle'] = wrapTo2pi(circularDistance(maleParams['heading'].interpolate(), leftW)) - np.pi
            maleParams['rightWingAngle'] = wrapTo2pi(circularDistance(maleParams['heading'].interpolate(), rightW)) - np.pi
            maleParams['interWingDistance'] = getBodypartDistance(malePositions, 'wingRight', 'wingLeft')
            maleParams['abdomenLength'] = getBodypartDistance(malePositions, 'abdomenTop', 'genitalia')
            maleParams['abdomenWidth'] = getBodypartDistance(malePositions, 'abdomenLeft', 'abdomenRight')
            maleParams['abdomentipdirection'] = getBodypartAngle(malePositions, 'genitalia', 'abdomenTop')
            maleParams['abdomentipOffset'] = circularDistance(maleParams['abdomentipdirection'],maleParams['heading'])
            abdomen_lower_width = getBodypartDistance(malePositions, 'abdomenLowerLeft', 'abdomenLowerRight')
            maleParams['abdomenLowerwidth'] = abdomen_lower_width
            maleParams['abdomenWidthRatio'] = abdomen_lower_width / maleParams['abdomenWidth']
            maleParams['abdomentipdirection'] = getBodypartAngle(malePositions, 'genitalia', 'abdomenTop')
            maleParams['abdomentipOffset'] = circularDistance(maleParams['abdomentipdirection'],maleParams['heading'])
            ableft = getBodypartAngle(malePositions, 'genitalia','abdomenLowerLeft')
            abright = getBodypartAngle(malePositions, 'genitalia','abdomenLowerRight')
            abmidline = getBodypartAngle(malePositions, 'genitalia', 'abdomenCenter')
            abtip_angle = wrapTo2pi(circularDistance(abright,ableft))
            maleParams['abdomen_tip_angle'] = abtip_angle
            abtip_left = wrapTo2pi(circularDistance(abmidline,ableft))
            abtip_right = wrapTo2pi(circularDistance(abright,abmidline))
            abtip_offset = abs(abtip_left - abtip_right)
            maleParams['abdomentipOffset_m2'] = abtip_offset

            midleg_left_1 = getBodypartAngle(malePositions, 'midlegLeftJoint1', 'thoraxCenter')
            midleg_right_1 = getBodypartAngle(malePositions, 'midlegRightJoint1', 'thoraxCenter')
            midleg_left_2 = getBodypartAngle(malePositions, 'midlegLeftJoint1', 'midlegLeftJoint2')
            midleg_right_2 = getBodypartAngle(malePositions, 'midlegRightJoint1', 'midlegRightJoint2')

            midleg_left_Joint1_Ang = wrapTo2pi(circularDistance(midleg_left_1,midleg_left_2))
            midleg_right_Joint1_Ang = wrapTo2pi(circularDistance(midleg_right_1,midleg_right_2))
            maleParams['midleg_left_Joint1_Ang'] = midleg_left_Joint1_Ang
            maleParams['midleg_right_Joint1_Ang'] = midleg_right_Joint1_Ang

            midleg_left_2p = getBodypartAngle(malePositions, 'midlegLeftJoint2', 'midlegLeftJoint1')
            midleg_right_2p = getBodypartAngle(malePositions, 'midlegRightJoint2', 'midlegRightJoint1')
            midleg_left_3 = getBodypartAngle(malePositions, 'midlegLeftJoint2', 'midlegLeft')
            midleg_right_3 = getBodypartAngle(malePositions, 'midlegRightJoint2', 'midlegRight')

            midleg_left_Joint2_Ang = wrapTo2pi(circularDistance(midleg_left_2p,midleg_left_3))
            midleg_right_Joint2_Ang = wrapTo2pi(circularDistance(midleg_right_2p,midleg_right_3))
            maleParams['midleg_left_Joint2_Ang'] = midleg_left_Joint2_Ang
            maleParams['midleg_right_Joint2_Ang'] = midleg_right_Joint2_Ang

            hindleg_left_1 = getBodypartAngle(malePositions, 'hindlegLeftJoint1', 'thoraxCenter')
            hindleg_right_1 = getBodypartAngle(malePositions, 'hindlegRightJoint1', 'thoraxCenter')
            hindleg_left_2 = getBodypartAngle(malePositions, 'hindlegLeftJoint1', 'hindlegLeftJoint2')
            hindleg_right_2 = getBodypartAngle(malePositions, 'hindlegRightJoint1', 'hindlegRightJoint2')

            hindleg_left_Joint1_Ang = wrapTo2pi(circularDistance(hindleg_left_1,hindleg_left_2))
            hindleg_right_Joint1_Ang = wrapTo2pi(circularDistance(hindleg_right_1,hindleg_right_2))
            maleParams['hindleg_left_Joint1_Ang'] = hindleg_left_Joint1_Ang
            maleParams['hindleg_right_Joint1_Ang'] = hindleg_right_Joint1_Ang

            hindleg_left_2p = getBodypartAngle(malePositions, 'hindlegLeftJoint2', 'hindlegLeftJoint1')
            hindleg_right_2p = getBodypartAngle(malePositions, 'hindlegRightJoint2', 'hindlegRightJoint1')
            hindleg_left_3 = getBodypartAngle(malePositions, 'hindlegLeftJoint2', 'hindlegLeft')
            hindleg_right_3 = getBodypartAngle(malePositions, 'hindlegRightJoint2', 'hindlegRight')

            hindleg_left_Joint2_Ang = wrapTo2pi(circularDistance(hindleg_left_2p,hindleg_left_3))
            hindleg_right_Joint2_Ang = wrapTo2pi(circularDistance(hindleg_right_2p,hindleg_right_3))
            maleParams['hindleg_left_Joint2_Ang'] = hindleg_left_Joint2_Ang
            maleParams['hindleg_right_Joint2_Ang'] = hindleg_right_Joint2_Ang

            # get inter-fly distance based on centroid
            IFD = np.sqrt(np.sum(np.square(maleCenter - femaleCenter),axis=1))
            femaleParams['dist2Other'] = IFD[:copFrame]
            femaleParams['facingAngle'], femaleParams['angBetween'] = getRelativeOrientations(femaleParams, maleParams)

            maleParams['dist2Other'] = IFD[:copFrame]
            maleParams['facingAngle'], maleParams['angBetween'] = getRelativeOrientations(maleParams, femaleParams)

            #write files
            femaleParams.to_excel(os.path.join(pathname, filename.replace('.h5', '_femaleJAABA.xlsx')))
            maleParams.to_excel(os.path.join(pathname, filename.replace('.h5', '_maleJAABA.xlsx')))

C:\Users\rutalab\AppData\Local\Temp\ipykernel_28580\163791931.py:20: RuntimeWarning:

Mean of empty slice

C:\Users\rutalab\AppData\Local\Temp\ipykernel_28580\163791931.py:20: RuntimeWarning:

Mean of empty slice

C:\Users\rutalab\AppData\Local\Temp\ipykernel_28580\163791931.py:48: RuntimeWarning:

invalid value encountered in remainder

C:\Users\rutalab\AppData\Local\Temp\ipykernel_28580\163791931.py:48: RuntimeWarning:

invalid value encountered in remainder

C:\Users\rutalab\AppData\Local\Temp\ipykernel_28580\163791931.py:39: RuntimeWarning:

invalid value encountered in true_divide

C:\Users\rutalab\AppData\Local\Temp\ipykernel_28580\163791931.py:48: RuntimeWarning:

invalid value encountered in remainder

C:\Users\rutalab\AppData\Local\Temp\ipykernel_28580\163791931.py:39: RuntimeWarning:

invalid value encountered in true_divide

C:\Users\rutalab\AppData\Local\Temp\ipykernel_28580\163791931.py:48: RuntimeWarning:

invalid value encountered in remainder

C:\Users\rutalab\AppData\L

In [ ]:

            df = femaleParams
            df2 = maleParams
            fig = make_subplots(specs=[[{"secondary_y": True}]])

            #add angles
            fig.add_scatter(y=df['abdomenLength'], mode='lines',
                            hovertext=df['abdomenLength'], hoverinfo="text",
                            name='female abdomen length (px)', secondary_y=False)

            #add angles
            fig.add_scatter(y=df2['abdomenLength'], mode='lines',
                            hovertext=df['abdomenLength'], hoverinfo="text",
                            name='male abdomen length (px)', secondary_y=False)


            fig.show()